In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
# Item Rando:
# 26 'randomizeable' items
# 100 'randomizeable' drop locations
# idea 1: location : item_id
# ~ 500 bits (locations * log(items))
# idea 2: item : location_id
# ~ 700 bits (locations * log(locations))
# idea 3: location : common_item_id
# major_item : location_id
# 300 bits + 126 bits

In [3]:
# Door Rando:
# ~550 doors
# idea 1: door: door_id
# 550 * log(550) = 5500 bits
# idea 2: left_door : right_door_id
# top_door : bottom_door_id
# 225 * log(225) = 2040
# idea 3:
# X, Y
# overworld is ~ 128 x 128
# 14 bits per room * 255 rooms = 3570

In [4]:
import sys
sys.path.append("../..")

In [5]:
from encoding.parse_rooms import parse_rooms, parse_exits, dictify_rooms

In [6]:
from functools import cache
from itertools import combinations

In [7]:
rooms = parse_rooms("../../encoding/dsl/rooms_for_alloy.txt")
exits = parse_exits("../../encoding/dsl/exits_for_alloy.txt")
design = dictify_rooms(rooms, exits)

In [8]:
from rom_tools.rom_manager import RomManager
from abstraction_validation.sm_paths import *

In [9]:
from bdds.bdd_core import *
from bdds.node_bdds import *

In [10]:
from functools import reduce

In [11]:
all_nodes = []
for r, room in rooms.items():
    for node in room.graph.name_node.keys():
        all_nodes.append(node)
node_ids = {n:i for i,n in enumerate(all_nodes)}

In [12]:
# Issue with design translation via chopping off the last _
node_ids["Spore_Spawn_Spawn"] = node_ids["Spore_Spawn_Spore_Spawn"]
node_ids["Golden_Torizo_Torizo"] = node_ids["Golden_Torizo_Golden_Torizo"]
node_ids["Mother_Brain_Brain"] = node_ids["Mother_Brain_Mother_Brain"]

In [13]:
major_items = ["MB", "B", "SPB", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "PLB", "Spazer", "XR", "IB", "SJ"]
minor_items = ["M", "S", "PB", "E", "RT"]
rando_items = ["B", "MB", "PB", "SPB", "S", "M", "G", "SA", "V", "GS", "SB", "HJ", "CB", "WB", "E", "PLB", "Spazer", "RT", "XR", "IB", "SJ"]
assert set(rando_items) == set(major_items) | set(minor_items)

In [14]:
context = mk_context_id(node_ids)

In [15]:
from functools import reduce

In [16]:
def reduce_and(clauses):
    return reduce(lambda x, y: x & y, clauses, context.true)

def reduce_or(clauses):
    return reduce(lambda x, y: x | y, clauses, context.false)

In [17]:
def loc_id(room_name, node_name, when="prev"):
    node_id = node_ids[f"{room_name}_{node_name}"]
    return context.add_expr(f"node_id_{when} = {node_id}")

@cache
def item_transitions(item_gained=None):
    if item_gained is None:
        return context.add_expr("(items_unchanged)", with_ops = True)
    clauses = []
    for i in design["Items"] | design["Bosses"]:
        if i == item_gained:
            clause = context.add_expr(f"{i}_prev < {i}_next")
        else:
            clause = context.add_expr(f"{i}_prev = {i}_next")
        clauses.append(clause)
    return reduce_and(clauses)

def itemset_to_bdd(itemset):
    if len(itemset) == 0:
        return context.true
    else:
        return reduce_and([context.add_expr(f"{item}_prev = 1") for item in itemset])

def required_itemsets(itemsets):
    return reduce_or([itemset_to_bdd(itemset) for itemset in itemsets])

def rando_transitions(room_name, node_name, family, possible_items):
    t = context.false
    node_name = room_name + "_" + node_name
    drop_name = f"{family}_drop_{node_name}"
    for i,item in enumerate(possible_items):
        t |= context.add_expr(f"{drop_name} = {i}") & item_transitions(item)
    return t

In [18]:
major_nodes = []
minor_nodes = []
other_drop_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        node_str = room_name + "_" + node_name
        if d in major_items:
            major_nodes.append(node_str)
        elif d in minor_items:
            minor_nodes.append(node_str)
        else:
            other_drop_nodes.append(node_str)

In [19]:
assert len(major_nodes) == len(major_items)
assert len(minor_nodes) == 100 - len(major_nodes)

In [20]:
drop_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [21]:
assert len(drop_nodes) == 100

In [22]:
major_item_nodes = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        if d in rando_items:
            drop_nodes.append(room_name + "_" + node_name)

In [23]:
#limit design freedom by setting a limited number of locations
#drop_nodes = ["Bomb_Torizo_Bombs", "Big_Pink_M2"]
n_major_rando = 16
n_minor_rando = 0
major_drop_nodes = major_nodes[:n_major_rando]
minor_drop_nodes = minor_nodes[:n_minor_rando]
#drop_nodes = []

In [24]:
design_vars = {
    **{ f"major_drop_{node}": (0,len(major_items)-1) for node in major_drop_nodes },
    **{ f"minor_drop_{node}": (0,len(minor_items)-1) for node in minor_drop_nodes }
}

In [25]:
if len(design_vars) > 0:
    context.declare(**design_vars)

In [26]:
# Ensure uniqueness and validity
validity = context.true
for node in major_drop_nodes:
    validity &= context.add_expr(f"major_drop_{node} < {len(major_items)}")
for node in minor_drop_nodes:
    validity &= context.add_expr(f"minor_drop_{node} < {len(major_items)}")

# Only drop one thing
# Sum is faster to compute than O(n^2) !=s
uniqueness = context.true
for i,item in enumerate(major_items):
    uniqueness &= context.add_expr(" + ".join([f"ite(major_drop_{node} = {i},1,0)" for node in major_drop_nodes] + ["0"]) + " <= 1")

valid_design = validity & uniqueness

In [27]:
original_design = context.true
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        name = room_name + "_" + node_name
        if d in major_items:
            if name in major_drop_nodes:
                original_design &= context.add_expr(f"major_drop_{name} = {major_items.index(d)}")
        if d in minor_items:
            if name in minor_drop_nodes:
                original_design &= context.add_expr(f"minor_drop_{name} = {minor_items.index(d)}")

In [28]:
edits = []
for room_name, room in design["Rooms"].items():
    for node_name, d in room["Drops"].items():
        name = room_name + "_" + node_name
        if d in major_items:
            if name in major_drop_nodes:
                edits.append(f"ite(major_drop_{name} = {major_items.index(d)},0,1)")
        if d in minor_items:
            if name in minor_drop_nodes:
                edits.append(f"ite(minor_drop_{name} = {minor_items.index(d)},0,1)")
all_edits = " + ".join(edits)
max_distance = 4
edit_distance = context.add_expr(all_edits + f" <= {max_distance}")
edit_distance_equal = context.add_expr(all_edits + f" = {max_distance}")

In [29]:
#TODO: minor items
def show_design(pick):
    for v,id in pick.items():
        print(f"{v}: {major_items[id]}")

In [30]:
from tqdm import tqdm

In [31]:
# Build individual BDDs
room_bdds = []
door_bdds = []
for room_name, room in tqdm(design['Rooms'].items()):
    links = []
    for node_name in room['Nodes']:
        s = loc_id(room_name, node_name) & loc_id(room_name, node_name, when="next") & item_transitions()
        links.append(s)
        if node_name in room['Drops']:
            if room_name + "_" + node_name in major_drop_nodes:
                s = loc_id(room_name, node_name) & loc_id(room_name, node_name, when="next") & rando_transitions(room_name, node_name, "major", major_items)
            elif room_name + "_" + node_name in minor_drop_nodes:
                s = loc_id(room_name, node_name) & loc_id(room_name, node_name, when="next") & rando_transitions(room_name, node_name, "minor", minor_items)
            else:
                s = loc_id(room_name, node_name) & loc_id(room_name, node_name, when="next") & item_transitions(room['Drops'][node_name])
            links.append(s)
    for node_name, door in room['Doors'].items():
        d = loc_id(room_name, node_name) & loc_id(door['Room'], door['Node'], when="next") & item_transitions()
        door_bdds.append(d)
    for node_name, edges in room['Edges'].items():
        for edge in edges:
            other_node_name = edge['Terminal']
            s = loc_id(room_name, node_name) & loc_id(room_name, other_node_name, when="next") & required_itemsets(edge['Requirements']) & item_transitions()
            links.append(s)
    room_bdd = reduce_or(links)
    #print(room_bdd.count())
    room_bdds.append(room_bdd)

100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [00:01<00:00, 197.28it/s]


In [32]:
doors_bdd = reduce_or(door_bdds)
rooms_bdd = reduce_or(room_bdds)
trans = doors_bdd | rooms_bdd

In [33]:
# Compute the design UNCONDITIONAL item-item reachability transes

In [34]:
items_unchanged = context.add_expr("items_unchanged", with_ops=True)
item_nexts = [k for k in item_vars if k.endswith("_next")]
item_prevs = [k for k in item_vars if k.endswith("_prev")]

In [35]:
# Find items that do not affect reachability:
bad_items = [
    # 'Implied' items
    "Botwoon",
    "Spore_Spawn",
    "Golden_Torizo",
    "Drain",
    "Shaktool",
    # Items that don't impact reachability
    "Spazer",
    "XR",
    "RT",
]
nuisances = [f"{i}_prev" for i in bad_items] + [f"{i}_next" for i in bad_items]
#bad_items = []
#for i in design["Items"] | design["Bosses"]:
#    istr = f"{i}_prev"
#    i_without = context.add_expr(f"{istr} = 0")
#    i_with = context.add_expr(f"{istr} = 1")
#    if context.exist([istr], (trans & i_without)) == context.exist([istr], (trans & i_with)):
#        bad_items.append(i)

In [36]:
trans = context.exist(nuisances, trans)

In [39]:
list(design_vars.keys())

['major_drop_Bomb_Torizo_Bombs',
 'major_drop_Big_Pink_CB',
 'major_drop_Morph_Ball_Room_MB',
 'major_drop_XRay_XR',
 'major_drop_Spazer_Spazer',
 'major_drop_Varia_Suit_V',
 'major_drop_Ice_Beam_IB',
 'major_drop_Hi_Jump_HJ',
 'major_drop_Grapple_G',
 'major_drop_Speed_Booster_SB',
 'major_drop_Wave_Beam_WB',
 'major_drop_Screw_Attack_SA',
 'major_drop_Gravity_GS',
 'major_drop_Plasma_PLB',
 'major_drop_Spring_Ball_SPB',
 'major_drop_Space_Jump_SJ']

In [54]:
duc_trans = context.forall(design_vars.keys(), trans)

In [55]:
dc_trans = context.exist(design_vars.keys(), trans) & ~design_unconditional_trans

In [59]:
duc_trans.dag_size, dc_trans.dag_size, trans.dag_size

(10928, 354, 22139)

In [61]:
def find_closure_sbfs(trans, start_bdd):
    n = 0
    closure = start_bdd & trans
    closure_last = context.false
    trans_tn = context.let(prev_to_temp, trans)
    while closure != closure_last:
        closure_last = closure
        closure |= context.exist(temps, context.let(next_to_temp, closure) & trans_tn)
        print(n, closure.dag_size)
        n += 1
    return closure

In [63]:
%%time
# Restrict endpoints to only important nodes
trans_important = find_closure_sbfs(duc_trans, important_prev) & context.let(prev_to_next, important_prev)

0 21402
1 38028
2 53745
3 75897
4 97355
5 131531
6 164662
7 223805
8 268501
9 355430
10 418518
11 523933
12 607748
13 757849
14 863021
15 1060127


KeyboardInterrupt: 

In [47]:
def find_reachable_sbfs(trans, start_bdd):
    n = 0
    reachable = start_bdd
    reachable_last = context.false
    while reachable != reachable_last:
        reachable_last = reachable
        reachable |= context.let(next_to_prev, context.exist(prevs, reachable & trans))
        print(n, reachable.dag_size)
        n += 1
    return reachable

In [48]:
from data_types import item_set

In [49]:
def mk_itemset_expr(itemset, when="prev"):
    clauses = []
    for i in item_mapping:
        if i in itemset:
            clause = context.add_expr(f"{i}_{when} = 1")
        else:
            clause = context.add_expr(f"{i}_{when} = 0")
        clauses.append(clause)
    return reduce_and(clauses)

In [50]:
def find_reachable_special(start_bdd):
    n = 0
    reachable = start_bdd
    reachable_last = context.false
    while reachable != reachable_last:
        reachable_last = reachable
        # one teleport step
        reachable |= context.let(next_to_prev, context.exist(prevs, reachable & trans_important))
        # one item step
        reachable |= context.let(next_to_prev, context.exist(prevs, reachable & trans_item))
        print(n, reachable.dag_size)
        n += 1
    return reachable

In [51]:
item_next_to_prev = {k:v for k,v in next_to_prev.items() if k != "node_id_next"}

In [71]:
def find_reachable_special2(start_bdd):
    n = 0
    trans_important_partial = context.exist(item_nexts, trans_important)
    trans_item_partial = context.exist(["node_id_next"], trans_item)
    reachable = start_bdd
    reachable_last = context.false
    while reachable != reachable_last:
        reachable_last = reachable
        # one teleport step
        reachable |= context.let({"node_id_next": "node_id_prev"}, context.exist(["node_id_prev"], reachable & trans_important_partial))
        # one item step
        reachable |= context.let(item_next_to_prev, context.exist(item_prevs, reachable & trans_item_partial))
        print(n, reachable.dag_size)
        n += 1
    return reachable

In [90]:
#TODO: figure out a principled stopping condition
def find_completable_bidirectional(start_bdd, end_bdd, max_iterations):
    reachable_forward = start_bdd
    reachable_backward = end_bdd
    for t in range(max_iterations):
        reachable_forward |= context.let(next_to_prev, context.exist(prevs, reachable_forward & teleport_trans))
        reachable_backward |= context.let(prev_to_next, context.exist(nexts, reachable_backward & teleport_trans))
        print(t, reachable_forward.dag_size, reachable_backward.dag_size)
    return reachable_forward & context.let(next_to_prev, reachable_backward)

In [72]:
start_bdd = context.add_expr("node_id_prev = 0") & mk_itemset_expr(item_set.ItemSet())

In [88]:
end_bdd = context.add_expr("node_id_next = 1")

In [93]:
%%time
#reachable = find_reachable_sbfs(teleport_trans, start_bdd)
#reachable = find_reachable_special2(start_bdd & valid_design & edit_distance)
reachable = find_completable_bidirectional(start_bdd, end_bdd, 17)

0 54 2016
1 392 4676
2 412 53281
3 424 192465
4 458 353001
5 746 2137110
6 744 2050716


KeyboardInterrupt: 

In [82]:
end_reachable = context.add_expr("node_id_prev = 1")

In [83]:
designs = context.exist(nexts + prevs, reachable)

In [84]:
designs.count()

2331933.0

In [85]:
good_designs = context.exist(nexts + prevs, reachable & end_reachable)

In [86]:
good_designs.count()

76325.0

In [60]:
context.support(good_designs)

{'major_drop_Big_Pink_CB',
 'major_drop_Bomb_Torizo_Bombs',
 'major_drop_Grapple_G',
 'major_drop_Gravity_GS',
 'major_drop_Hi_Jump_HJ',
 'major_drop_Ice_Beam_IB',
 'major_drop_Morph_Ball_Room_MB',
 'major_drop_Plasma_PLB',
 'major_drop_Screw_Attack_SA',
 'major_drop_Space_Jump_SJ',
 'major_drop_Spazer_Spazer',
 'major_drop_Speed_Booster_SB',
 'major_drop_Spring_Ball_SPB',
 'major_drop_Varia_Suit_V',
 'major_drop_Wave_Beam_WB',
 'major_drop_XRay_XR'}